# Mapping and OpenStreetMaps in Python
In the first part of the tutorial, we look at how we can easily create maps in Python using `folium`. In the second half, we populate these maps with OpenStreetMap (OSM) data.

## Creating Maps with folium

In [ ]:
import folium

In [ ]:
lat_lon_westerberg = [52.286, 8.023]

We can create a map using `folium` simply by specifying the center location of the map. By default it will render a map using tiles from OpenStreetMap.

In [ ]:
m = folium.Map(location=lat_lon_westerberg, zoom_start=20)
m

We can also use other tiles, e.g. a terrain map, if we want to.

In [ ]:
m = folium.Map(location=lat_lon_westerberg, zoom_start=10, tiles='Stamen Terrain')
m

We can put markers on our map.

In [ ]:
m = folium.Map(location=lat_lon_westerberg, zoom_start=20)

folium.Marker(lat_lon_westerberg).add_to(m)

m

And we can customize these markers using popups, tooltips and custom icons.

In [ ]:
m = folium.Map(location=lat_lon_westerberg, zoom_start=20)

folium.Marker(lat_lon_westerberg, 
              popup='CodeForOsnabrück', 
              tooltip='Click me!',
              icon=folium.Icon(color='blue', icon='code', prefix='fa')).add_to(m)

m

We can also put complete plots on our markers.

In [ ]:
import pandas as pd
import numpy as np
import altair.vegalite.v1 as alt

data = pd.DataFrame(data={
    'open data': np.arange(10),
    'awesomeness': np.exp(np.arange(10))
})

data



In [ ]:
chart = alt.Chart(data).mark_point().encode(
    x='open data',
    y='awesomeness'
)

chart

In [ ]:
m = folium.Map(location=lat_lon_westerberg, zoom_start=20)

folium.Marker(location=lat_lon_westerberg, 
              tooltip='Click me!',
              icon=folium.Icon(color='blue', icon='code', prefix='fa'),
              popup=folium.Popup(max_width=450).add_child(
                  folium.VegaLite(chart, width='100%', height='100%'))
             ).add_to(m)

m

## OpenStreetMap data with Overpass
Overpass is the OpenStreetMap API for quering data. It provides its own query language (Overpass QL) and access to all the data stored in OSM. While Overpass offers many possibilities to retrieve ways and relations, we will only look at ways to retrieve nodes. Those are points on the map with a specific location and specific attributes.
http://overpass-turbo.eu/

For retrieving nodes, Overpass QL is quite straitghforward.
```
node[attribute="value"];
```

gets all nodes that have a certain value for a certain attribute. Each statement has to end in a semicolon. One can form an `AND` connection over several attributes by

```
node
    [attribute1="value1"]
    [attribute2="value1"];
```

It is important to filter the result, based on some geographical area. Otherwise we would search on the whole world, which could take some time and might be quite a lot of data. To restrict the query to an area, we pass a reference to that area in parenthesis at the end of the statement. This could either be a comma, separated list of longitudes and latitudes like `(50.745,7.17,50.75,7.18)`. This values represent *southern-most latitude, western-most longitude, northern-most latitude, eastern-most longitude*, in that order. More convenient for us, we can reference the area of Osnabrück by its OSM id like `(area:3600062631)`.

```
node
    [attribute1="value1"]
    (area:3600062631);
```


Moreover, one can form the union of the results of several statements, by wrapping them in parenthesis.

```
(
node[attribute1="value1"](area:3600062631);
node[attribute2="value1"](area:3600062631);
);
```

For more advanced options, see the language guide https://wiki.openstreetmap.org/wiki/Overpass_API/Language_Guide.

### Working with Overpass in Python
To access Overpass from inside Python, we need the `requests` module to send, well requests, to the API.

In [ ]:
import requests

api_url = 'https://overpass-api.de/api/interpreter/'

We simply send our query as `POST` data to the API URL. It is important to specify `[out:json]` at the beginning, so we get the data back as JSON which is much easier to work with than XML.

In [ ]:
query = '''
[out:json];
node
    [amenity="school"]
    (area:3600062631);
out;
'''

r = requests.post(api_url, data=query)
overpass_response = r.json()
overpass_response

To find a suitable center for the map, we calculate the mean longitude and latitude of all the nodes that matched our query.

In [ ]:
import numpy as np

latitudes = [element['lat'] for element in overpass_response['elements']]
longitudes = [element['lon'] for element in overpass_response['elements']]
mean_lat = np.mean(latitudes)
mean_lon = np.mean(longitudes)
mean_lon, mean_lat

Next we create a map centered at that point.

In [ ]:
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=15)
m

Now we want to put the nodes that we got from overpass on our map. For that we first have to convert our data to common exchange format for geodata called `GeoJSON`.

In [ ]:
geojson = {
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "Point",
            "coordinates": [d["lon"], d["lat"]],
            },
        "properties" : d['tags'],
     } for d in overpass_response['elements']]
}

geojson


Now we can add markers to our map.

In [ ]:
folium.GeoJson(
    geojson, 
    tooltip=folium.GeoJsonTooltip(fields=['name'], sticky=False)).add_to(m)

m

So we do not always have to through all these steps, we break down our workflow into reusable functions, so we can get a map directly from an Overpass query. 

In [ ]:
def query_to_map(query):
    overpass_response = query_to_response(query)
    center = get_center(overpass_response)
    geojson = overpass_response_to_geojson(overpass_response)
    m = geojson_to_map(center, geojson)
    return m
    
    
def query_to_response(query):
    response = requests.post(api_url, data=query)
    overpass_response = response.json()
    return overpass_response
  
    
def overpass_response_to_geojson(overpass_response):
    geojson = {
        "type": "FeatureCollection",
        "features": [
        {
            "type": "Feature",
            "geometry" : {
                "type": "Point",
                "coordinates": [d["lon"], d["lat"]],
                },
            "properties" : d['tags'],
         } for d in overpass_response['elements']]
    }
    return geojson


def geojson_to_map(center, geojson):
    m = folium.Map(location=[mean_lat, mean_lon], zoom_start=13)
    folium.GeoJson(
            geojson, 
            tooltip=folium.GeoJsonTooltip(fields=['name'], sticky=False)).add_to(m)
    return m


def get_center(overpass_response):
    latitudes = [element['lat'] for element in overpass_response['elements']]
    longitudes = [element['lon'] for element in overpass_response['elements']]
    mean_lat = np.mean(latitudes)
    mean_lon = np.mean(longitudes)
    return [mean_lon, mean_lat]

In [ ]:
query = '''
[out:json];
node
    [amenity="school"]
    (area:3600062631);
out;
'''
query_to_map(query)

In [ ]:
query = '''
[out:json];
node
    [amenity="theatre"]
    (area:3600062631);
out;
'''
query_to_map(query)

One of the most useful keys to find nodes of interest is `amenity`. See https://wiki.openstreetmap.org/wiki/Key:amenity for a full reference of the values it can take. Now its time for you to try out your own queries!

In [ ]:
# Create your own maps here.

For more advanced geospatial anaylsis there are modules like geopandas and PySAL, but where not going into the details here.

In [ ]:
import geopandas
import json

In [ ]:
with open('response.geojson', 'w') as fp:
    json.dump(geojson, fp)

In [ ]:
data = geopandas.read_file('response.geojson')
data.head()

In [ ]:
data.plot()

## Where to go from here?

The planet is the dataset! Other cool things you can do with OpenStreetMap.

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('n_qXOqtarpc')

Geospatial data analysis in Python. A full tutorials, by the core developers of the libraries.

In [ ]:
YouTubeVideo('kJXUUO5M4ok')